In [1]:
!pip install tensorflow opencv-python mediapipe scikit-learn matplotlib

^C


In [1]:
pip install --upgrade mediapipe


^C
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install opendatasets

Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install --no-warn-script-location slugify kaggle

In [44]:
import opendatasets as od

In [45]:
dataset = 'https://www.kaggle.com/datasets/risangbaskoro/wlasl-processed/data'

In [46]:
od.download(dataset)

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username:

  arwasayed


Your Kaggle Key:

  ········


Dataset URL: https://www.kaggle.com/datasets/risangbaskoro/wlasl-processed


100%|█████████████████████████████████████████████████████████████████████████████| 4.82G/4.82G [41:44<00:00, 2.07MB/s]


In [10]:
with open('wlasl-processed/WLASL_v0.3.json', 'r') as json_file:
    all_data = json.load(json_file)

In [18]:
import pandas as pd
main_path = "wlasl-processed/"
wlasl_df = pd.read_json(main_path + "WLASL_v0.3.json")
wlasl_df.head(20)

,gloss,instances
0,book,"[{'bbox': [385, 37, 885, 720], 'fps': 25, 'fra..."
1,drink,"[{'bbox': [551, 68, 1350, 1080], 'fps': 25, 'f..."
2,computer,"[{'bbox': [0, 0, 360, 240], 'fps': 25, 'frame_..."
3,before,"[{'bbox': [0, 0, 360, 240], 'fps': 25, 'frame_..."
4,chair,"[{'bbox': [0, 0, 360, 240], 'fps': 25, 'frame_..."
5,go,"[{'bbox': [0, 0, 360, 240], 'fps': 25, 'frame_..."
6,clothes,"[{'bbox': [0, 0, 360, 240], 'fps': 25, 'frame_..."
7,who,"[{'bbox': [0, 0, 360, 240], 'fps': 25, 'frame_..."
8,candy,"[{'bbox': [0, 0, 360, 240], 'fps': 25, 'frame_..."
9,cousin,"[{'bbox': [134, 17, 491, 370], 'fps': 25, 'fra..."


In [12]:
all_data.gloss()

AttributeError: 'list' object has no attribute 'gloss'

In [1]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp
import json
from tqdm import tqdm

In [2]:
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils


In [3]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [4]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS) # Draw face connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Draw pose connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw right hand connections

In [5]:
def draw_styled_landmarks(image, results):
    # Draw face connections
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS,
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             )
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             )
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             )
    # Draw right hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             )

In [6]:
mp_drawing.draw_landmarks??

Signature:
mp_drawing.draw_landmarks(
    image: numpy.ndarray,
    landmark_list: mediapipe.framework.formats.landmark_pb2.NormalizedLandmarkList,
    connections: Optional[List[Tuple[int, int]]] = None,
    landmark_drawing_spec: Union[mediapipe.python.solutions.drawing_utils.DrawingSpec, Mapping[int, mediapipe.python.solutions.drawing_utils.DrawingSpec]] = DrawingSpec(color=(0, 0, 255), thickness=2, circle_radius=2),
    connection_drawing_spec: Union[mediapipe.python.solutions.drawing_utils.DrawingSpec, Mapping[Tuple[int, int], mediapipe.python.solutions.drawing_utils.DrawingSpec]] = DrawingSpec(color=(224, 224, 224), thickness=2, circle_radius=2),
    is_drawing_landmarks: bool = True,
)
Source:   
def draw_landmarks(
    image: np.ndarray,
    landmark_list: landmark_pb2.NormalizedLandmarkList,
    connections: Optional[List[Tuple[int, int]]] = None,
    landmark_drawing_spec: Union[DrawingSpec,
                                 Mapping[int, DrawingSpec]] = DrawingSpec(
          

In [221]:
cap=cv2.VideoCapture(0)
# set mediapip model
with  mp_holistic.Holistic(min_detection_confidence=.5,min_tracking_confidence=0.5) as holistic :
    while cap.isOpened():
        # read feed
        ret,frame=cap.read()
        
        # make detection
        image,results=mediapipe_detection(frame,holistic)
        print(results)
        # Draw landmarks
        draw_styled_landmarks(image, results)
        # show screen
        cv2.imshow("open feed",image)
        if cv2.waitKey(10) & 0xff==ord('q'):
            break
            
    cap.release()
    cv2.destroyAllWindows()


<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

In [222]:
len(results.left_hand_landmarks.landmark)

21

In [223]:
results

mediapipe.python.solution_base.SolutionOutputs

In [224]:
mp_holistic.POSE_CONNECTIONS

frozenset({(0, 1),
           (0, 4),
           (1, 2),
           (2, 3),
           (3, 7),
           (4, 5),
           (5, 6),
           (6, 8),
           (9, 10),
           (11, 12),
           (11, 13),
           (11, 23),
           (12, 14),
           (12, 24),
           (13, 15),
           (14, 16),
           (15, 17),
           (15, 19),
           (15, 21),
           (16, 18),
           (16, 20),
           (16, 22),
           (17, 19),
           (18, 20),
           (23, 24),
           (23, 25),
           (24, 26),
           (25, 27),
           (26, 28),
           (27, 29),
           (27, 31),
           (28, 30),
           (28, 32),
           (29, 31),
           (30, 32)})

In [225]:
draw_landmarks(frame, results)

In [226]:
len(results.pose_landmarks.landmark)

33

In [227]:
# len(pos)

In [228]:
# len(lh)

In [7]:
def extract_keypoints(results):
    # pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    # return np.concatenate([pose, face, lh, rh])
    return np.concatenate([face, lh, rh])

In [8]:
# def extract_keypoints(results):
#     pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results[0].pose_landmarks.landmark]).flatten() if results[0].pose_landmarks else np.zeros(33*4)
#     face = np.array([[res.x, res.y, res.z] for res in results[0].face_landmarks.landmark]).flatten() if results[0].face_landmarks else np.zeros(468*3)
#     lh = np.array([[res.x, res.y, res.z] for res in results[0].left_hand_landmarks.landmark]).flatten() if results[0].left_hand_landmarks else np.zeros(21*3)
#     rh = np.array([[res.x, res.y, res.z] for res in results[0].right_hand_landmarks.landmark]).flatten() if results[0].right_hand_landmarks else np.zeros(21*3)
#     return np.concatenate([pose, face, lh, rh])

In [9]:
result_test = extract_keypoints(results)

NameError: name 'results' is not defined

In [357]:
np.save('0', result_test)

In [358]:
np.load('0.npy')

array([ 0.73264706,  0.48565644, -0.03464695, ...,  0.        ,
        0.        ,  0.        ])

In [10]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('MP_Data') 

# Actions that we try to detect
# actions = np.array(['hello', 'thanks', 'i love you','nice','to meet you','you are welcome','yes','no','please','sorry','i am','learn','sign language','excuse me' ,'name','what is','your','where are','you','from','how are you'])
actions = np.array(['hello','how are you','bad','fine','nice','yes','learn','bath','dad','food','stop','sad','I am','no thing','name','what is','your','to meet you','no','bed','mam','help','drink','sorry','thanks'])
# actions = np.array(['hello','how are you','bad','fine','nice','yes','learn','bath','dad','food'])
# Thirty videos worth of data
no_sequences = 30

# Videos are going to be 30 frames in length
sequence_length = 30

In [360]:
for action in actions: 
    for sequence in range(no_sequences):
        try: 
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

In [47]:
# video_dir = 'wlasl-processed/videos'
# data = [] # formatted data

# for i in tqdm(range(len(all_data)), ncols=100):
#     gloss = all_data[i]['gloss']
#     instances = all_data[i]['instances']
#     for instance in instances:
#         video_id = instance['video_id']
#         if os.path.exists(os.path.join(video_dir, f'{video_id}.mp4')):
#             video_path = os.path.join(video_dir, f'{video_id}.mp4')
#         else:
#             continue
            
#         frame_start = instance['frame_start']
#         frame_end = instance['frame_end']
#         split = instance['split']
#         data.append({
#             'gloss': gloss,
#             'video_path': video_path,
#             'frame_start': frame_start,
#             'frame_end': frame_end,
#             'split': split
#         })


100%|██████████████████████████████████████████████████████████| 2000/2000 [00:02<00:00, 996.00it/s]


In [34]:
data[:][0]

{'gloss': 'book',
 'video_path': 'wlasl-processed/videos\\69241.mp4',
 'frame_start': 1,
 'frame_end': -1,
 'split': 'train'}

In [52]:
cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    # NEW LOOP
    # Loop through actions
    for action in actions:
        # Loop through sequences aka videos
        for sequence in range(no_sequences):
            # Loop through video length aka sequence length
            for frame_num in range(sequence_length):
                ret, frame = cap.read()
    
                    # Make detections
                image, results = mediapipe_detection(frame, holistic)
                    #   print(results)
    
                    # Draw landmarks
                draw_styled_landmarks(image, results)
                    
                    # NEW Apply wait logic
                if frame_num == 0: 
                    cv2.putText(image, 'STARTING COLLECTION', (120,200), 
                                cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                        # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                    cv2.waitKey(50)
                else: 
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                        # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                    
                    # NEW Export keypoints
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)

                # Break gracefully
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
                    
    cap.release()
    cv2.destroyAllWindows()

In [30]:
# for video_info in data:
#     video_path = video_info['video_path']
#     gloss = video_info['gloss']
#     frame_start = video_info['frame_start']
#     frame_end = video_info['frame_end']
#     cap = cv2.VideoCapture(video_path)
#     with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
#         total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        
#         # Set the start and end frame based on the provided values or defaults
#         start_frame = frame_start if frame_start > 0 else 1
#         end_frame = frame_end if frame_end > 0 else total_frames
        
#         # Set the frame number to start from
#         cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)
        
#         for frame_num in range(start_frame, end_frame + 1):
#             # Read feed
#             ret, frame = cap.read()
            
#             if ret:
#                 # Make detections
#                 results = mediapipe_detection(frame,holistic)
#                 print (results)
#                 keypoints = extract_keypoints(results)
#                 npy_path = os.path.join(DATA_PATH, gloss, str(frame_num))
#                 np.save(npy_path, keypoints)
#             else:
#                 print(f"Error reading frame {frame_num} from video {video_path}")


(array([[[ 98,  97,  99],
        [ 98,  97,  99],
        [ 95,  97,  99],
        ...,
        [103, 100, 101],
        [105, 101, 104],
        [105, 101, 104]],

       [[ 98,  97,  99],
        [ 98,  97,  99],
        [ 95,  97,  99],
        ...,
        [103, 100, 101],
        [104, 100, 103],
        [105, 101, 104]],

       [[ 98,  97,  99],
        [ 98,  97,  99],
        [ 95,  97,  99],
        ...,
        [106, 101, 102],
        [106, 100, 103],
        [106, 100, 103]],

       ...,

       [[ 72,  70,  75],
        [ 73,  71,  76],
        [ 74,  73,  75],
        ...,
        [109, 103, 106],
        [109, 103, 106],
        [109, 103, 106]],

       [[ 72,  70,  75],
        [ 73,  71,  76],
        [ 76,  72,  75],
        ...,
        [109, 103, 106],
        [109, 103, 106],
        [109, 103, 106]],

       [[ 74,  72,  77],
        [ 74,  72,  77],
        [ 76,  72,  75],
        ...,
        [109, 103, 106],
        [109, 103, 106],
        [109, 103, 106]

AttributeError: 'tuple' object has no attribute 'pose_landmarks'

In [11]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [12]:
label_map = {label:num for num, label in enumerate(actions)}

In [13]:
label_map

{'hello': 0,
 'how are you': 1,
 'bad': 2,
 'fine': 3,
 'nice': 4,
 'yes': 5,
 'learn': 6,
 'bath': 7,
 'dad': 8,
 'food': 9,
 'stop': 10,
 'sad': 11,
 'I am': 12,
 'no thing': 13,
 'name': 14,
 'what is': 15,
 'your': 16,
 'to meet you': 17,
 'no': 18,
 'bed': 19,
 'mam': 20,
 'help': 21,
 'drink': 22,
 'sorry': 23,
 'thanks': 24}

In [14]:
sequences, labels = [], []
for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [15]:
x=np.array(sequences)

In [16]:
y = to_categorical(labels).astype(int)

In [288]:
print(y)

[[1 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 1]]


In [17]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

In [18]:
# X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.05)

In [50]:
y_test

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [51]:
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard
import os

In [52]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [53]:
from tensorflow import keras
from tensorflow.keras import layers

model = Sequential()
model.add(layers.Input(shape=(30,1530)))  # Define input shape here
model.add(layers.LSTM(64, return_sequences=True, activation='relu'))
model.add(layers.LSTM(128, return_sequences=True, activation='relu'))
model.add(layers.LSTM(64, return_sequences=False, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(actions.shape[0], activation='softmax'))


In [54]:
# model = Sequential()
# model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1662)))
# model.add(LSTM(128, return_sequences=True, activation='relu'))
# model.add(LSTM(64, return_sequences=False, activation='relu'))
# model.add(Dense(64, activation='relu'))
# model.add(Dense(32, activation='relu'))
# model.add(Dense(actions.shape[0], activation='softmax'))

In [55]:
x.shape

(750, 30, 1530)

In [56]:
res = [.7,.8,.9,.6,.8,.9,.7,.9,.5,.6,.8,.9,.7,.8,.9,.99,.88,.77,.66,.55,.48]

In [57]:
actions[np.argmax(res)]

'what is'

In [58]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [59]:
model.fit(X_train, y_train, epochs=1000, callbacks=[tb_callback])

Epoch 1/1000
19/19 ━━━━━━━━━━━━━━━━━━━━ 46s 147ms/step - categorical_accuracy: 0.0334 - loss: 3.4053
Epoch 2/1000
19/19 ━━━━━━━━━━━━━━━━━━━━ 2s 128ms/step - categorical_accuracy: 0.0326 - loss: 3.2137
Epoch 3/1000
19/19 ━━━━━━━━━━━━━━━━━━━━ 2s 94ms/step - categorical_accuracy: 0.0474 - loss: 3.1838
Epoch 4/1000
19/19 ━━━━━━━━━━━━━━━━━━━━ 2s 95ms/step - categorical_accuracy: 0.0547 - loss: 3.1167
Epoch 5/1000
19/19 ━━━━━━━━━━━━━━━━━━━━ 2s 107ms/step - categorical_accuracy: 0.1017 - loss: 3.0403
Epoch 6/1000
19/19 ━━━━━━━━━━━━━━━━━━━━ 2s 102ms/step - categorical_accuracy: 0.1289 - loss: 2.9523
Epoch 7/1000
19/19 ━━━━━━━━━━━━━━━━━━━━ 2s 95ms/step - categorical_accuracy: 0.1046 - loss: 2.9190
Epoch 8/1000
19/19 ━━━━━━━━━━━━━━━━━━━━ 2s 111ms/step - categorical_accuracy: 0.1314 - loss: 2.7562
Epoch 9/1000
19/19 ━━━━━━━━━━━━━━━━━━━━ 2s 99ms/step - categorical_accuracy: 0.1852 - loss: 2.5837
Epoch 10/1000
19/19 ━━━━━━━━━━━━━━━━━━━━ 2s 93ms/step - categorical_accuracy: 0.1884 - loss: 2.6747
Epo

KeyboardInterrupt: 

In [60]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm_3 (LSTM)                        │ (None, 30, 64)              │         408,320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_4 (LSTM)                        │ (None, 30, 128)             │          98,816 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_5 (LSTM)                        │ (None, 64)                  │          49,408 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 64)                  │           4,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 25)                  │             825 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,690,829 (6.45 MB)

 Trainable params: 563,609 (2.15 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 1,127,220 (4.30 MB)

In [61]:
res=model.predict(X_test)

5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 374ms/step


In [62]:
res

array([[5.7823945e-07, 4.8823128e-04, 1.6860531e-05, ..., 9.9136848e-08,
        4.5079489e-07, 1.0102744e-06],
       [1.0628005e-04, 1.9036730e-05, 2.2869578e-01, ..., 5.2209157e-06,
        1.5099780e-06, 2.7848879e-05],
       [1.5316170e-03, 1.5678857e-05, 8.6766999e-10, ..., 6.2225366e-01,
        2.6409753e-02, 1.7596696e-03],
       ...,
       [1.7548195e-04, 3.6871770e-05, 2.3583736e-01, ..., 3.6613815e-06,
        2.0334587e-06, 1.1503422e-05],
       [1.4060350e-07, 3.2660617e-03, 6.3988530e-08, ..., 3.6619866e-08,
        7.0102679e-08, 3.1173091e-08],
       [2.9504190e-06, 1.5502579e-02, 2.6928140e-03, ..., 7.7626103e-04,
        4.0055482e-05, 8.5201943e-09]], dtype=float32)

In [63]:
actions[np.argmax(res[0])]

'help'

In [64]:
actions[np.argmax(y_test)]

'name'

In [65]:
model.save_weights('action.weights.h5')

In [66]:
model.load_weights('action.weights.h5')

In [67]:
from sklearn.metrics import multilabel_confusion_matrix,accuracy_score

In [68]:
yhat=model.predict(X_train)

19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 33ms/step


In [69]:
ytrue =np.argmax(y_train,axis=1).tolist()
yhat=np.argmax(yhat,axis=1).tolist()

In [70]:
multilabel_confusion_matrix(ytrue,yhat)

array([[[571,   2],
        [  1,  26]],

       [[564,  14],
        [  1,  21]],

       [[575,   2],
        [  3,  20]],

       [[563,  13],
        [  4,  20]],

       [[570,   3],
        [  2,  25]],

       [[579,   0],
        [  6,  15]],

       [[573,   5],
        [  8,  14]],

       [[574,   3],
        [  2,  21]],

       [[574,   0],
        [  0,  26]],

       [[573,   2],
        [  7,  18]],

       [[570,   7],
        [  6,  17]],

       [[570,   6],
        [ 15,   9]],

       [[567,   9],
        [  2,  22]],

       [[562,  12],
        [  4,  22]],

       [[551,  23],
        [  4,  22]],

       [[572,   6],
        [ 17,   5]],

       [[564,  11],
        [  1,  24]],

       [[583,   0],
        [ 15,   2]],

       [[570,   4],
        [  4,  22]],

       [[575,   3],
        [  6,  16]],

       [[561,  13],
        [  9,  17]],

       [[573,   4],
        [  5,  18]],

       [[564,  11],
        [  6,  19]],

       [[572,   2],
        [ 14, 

In [71]:
multilabel_confusion_matrix??

Signature:
multilabel_confusion_matrix(
    y_true,
    y_pred,
    *,
    sample_weight=None,
    labels=None,
    samplewise=False,
)
Source:   
def multilabel_confusion_matrix(
    y_true, y_pred, *, sample_weight=None, labels=None, samplewise=False
):
    """Compute a confusion matrix for each class or sample.

    .. versionadded:: 0.21

    Compute class-wise (default) or sample-wise (samplewise=True) multilabel
    confusion matrix to evaluate the accuracy of a classification, and output
    confusion matrices for each class or sample.

    In multilabel confusion matrix :math:`MCM`, the count of true negatives
    is :math:`MCM_{:,0,0}`, false negatives is :math:`MCM_{:,1,0}`,
    true positives is :math:`MCM_{:,1,1}` and false positives is
    :math:`MCM_{:,0,1}`.

    Multiclass data will be treated as if binarized under a one-vs-rest
    transformation. Returned confusion matrices will be in the order of
    sorted unique labels in the union of (y_true, y_pred).

    Read mo

In [72]:
accuracy_score(ytrue,yhat)

0.7383333333333333

In [73]:
from scipy import stats

In [74]:
colors = [(245,117,16), (117,245,16), (16,117,245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        # cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

In [75]:
plt.figure(figsize=(18,18))
plt.imshow(prob_viz(res,actions,image,colors))

TypeError: only length-1 arrays can be converted to Python scalars

<Figure size 1800x1800 with 0 Axes>

In [76]:
sequence.append('abc')

In [77]:
sequence[-30:]

['abc']

In [78]:
sequence.append('def')

In [79]:
sequence.reverse()

In [80]:
sequence[-30:]

['def', 'abc']

In [81]:
np.unique(predictions[-10:])[0]

IndexError: index 0 is out of bounds for axis 0 with size 0

In [87]:
# 1. New detection variables
sequence = []
sentence = []
predictions = []
threshold = 0.5

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        # print(results)
        
        # # Draw landmarks
        # draw_styled_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            predictions.append(np.argmax(res))
            
            
        #3. Viz logic
            if np.unique(predictions[-30:])[0]==np.argmax(res): 
                if res[np.argmax(res)] > threshold: 
                    
                    if len(sentence) > 0: 
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                    else:
                        sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]

            # Viz probabilities
            # image = prob_viz(res, actions, image, colors)
            # image = prob_viz( actions, image, colors)
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
yes
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
yes
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
hello
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
yes
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
yes
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
yes
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
yes
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
hello
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
hello
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
hello
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
hello
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
hello
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
hello
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
hello
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
hello
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
hello
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
learn
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step
learn
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
nice
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
how are you
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
thanks
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
nice
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
n